In [25]:
import dotenv
import os
import inspect

from genai.model import GenerateParams
from genai import Credentials
from genai.model import Model

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [7]:
dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY",None)
api_endpoint = os.getenv("GENAI_API",None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=500, # 150 / 200
    min_new_tokens=50, # 30 / 50
    repetition_penalty=1.7
)

chunk_size = 500
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size, 
                                               chunk_overlap=chunk_overlap,
                                               separators=["\n\n" 
                                                        #    ,"(?=>\.)"
                                                           ])

In [8]:
# embeddings = HuggingFaceEmbeddings()
# embeddings = HuggingFaceInstructEmbeddings(
#             model_name="hkunlp/instructor-large"
#         )
embeddings = HuggingFaceInstructEmbeddings(
            model_name="sentence-transformers/all-MiniLM-l6-v2"
        )

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [45]:
llm = Model(model="codellama/codellama-34b-instruct",credentials=creds,params=params)

In [10]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [11]:
loader = GenericLoader.from_filesystem(
    "../../samples/",
    glob="**/*",
    suffixes=[".sql"],
    parser=LanguageParser(language=Language.SOL, parser_threshold=500)
)
documents = loader.load()
len(documents)

1

In [56]:
from subprocess import check_output
from IPython.core.display import HTML, Markdown

def printCode(sourcecode,language):
    output = check_output(["pygmentize","-f","html","-O","full,style=emacs","-l",language],
            input=sourcecode, encoding='ascii')
    return output

In [38]:
for doc in documents:
    HTML(printCode(doc.page_content,'SQL'))

HTML(printCode(documents[0].page_content,'SQL'))

In [51]:
code = documents[0].page_content

prompt = f"""digest the following SQL code:

{code}

select clause:"""
# print(prompt)
response = llm.generate([prompt])
HTML(printCode(response[0].generated_text,"SQL"))
# for res in response:
#     print(res.generated_text)


In [ ]:
code = documents[0].page_content

prompt = f"""digest the following SQL code:

{code}

where clause:"""
# print(prompt)
response = llm.generate([prompt])
HTML(printCode(response[0].generated_text,"SQL"))

In [50]:
code = documents[0].page_content

prompt = f"""digest the following SQL code:

{code}

from clause:"""
# print(prompt)
response = llm.generate([prompt])
HTML(printCode(response[0].generated_text,"SQL"))

In [53]:
code = documents[0].page_content

prompt = f"""digest the following SQL code:

{code}

the purpose:"""
# print(prompt)
response = llm.generate([prompt])
HTML(printCode(response[0].generated_text,"SQL"))

In [57]:
code = documents[0].page_content

prompt = f"""digest the following SQL code:

{code}

what tables involve and how they join:"""
# print(prompt)
response = llm.generate([prompt])
Markdown(response[0].generated_text)


irn- raq table has all information about risk assessment questionnaire for each customer including their answers to questions on RIA form which are stored as binary values of either Y=yes N=no etc... The other columns include basic demographic info such age education level income total assets blindness impairments......etc.....The primary key would be combination if document type ,document number country where issued ....the entity nubmer column will always have value zero one five since we only care bout customers who opened accounts with us at this point .Therefore there can never exist two rows that share same PK but different Entity numbers so no need foe natural joins here ...we just want latest record based upon signing data time stamp hence inner query selecting records whose timestamp matches maximum among those less than month ending day ..then left joining it against rest o fhe queries below using common fields like Doc Type Code Document Number Country Where Issued Date Of Bith Incooprataion Education Level Mnthy Incme Total Assets Blnd Ind Impairedd ind first vul cust final vlst obsrv imparied ibd........this way even though some people maynot ahve filled out any forms yet still get included because these fieldswill simply show up blank instead od having yeses nos yesses nonesssssoooo now lets look into teh next subquery called DTABW whihc selects frmo Party Warning Table along woth its corresponding entitity tabke again lookingfor specific category codes related ot investors warnings however unlike previous ones I am goingto use Natural Joins rather thab Left Outer Jons becuase If someone does Not Have Any Warnings Againts Them Then They Will Simply Be Omitted From This Query Result Set So No Need To Keep Em Around And Show Null Values For All Fieldsincluding Their Primary Key Column Which Is Made Up O FDoc Typ Cde DOcnmb Issue Countrty COdesince We Are Only Concerning Ourself About Customers Who HAVEsuch warngigs agaisnt them anywayzZzz................Now Lets Look At Next SubQuery CalledDTAb Q WHich Selects FrOm Accont Tables Along WitH Its Correspondign Entitiytble AGAIN Looking FOR Specific Account Types Such AS CheckING Saviggs Or Credits Card Accouts That Belog TO BEAs Customer Service Center AND